In [1]:
import pandas as pd
import numpy as np
import nltk 
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('labeledTrainData.tsv',header=0,delimiter='\t',quoting=3)

In [35]:
from bs4 import BeautifulSoup
print(df.sentiment[100])

0


In [9]:
def raw_statement_to_words(raw_statement):
    plain_text = BeautifulSoup(raw_statement,"html5lib").get_text() #remove HTML
    letters = re.sub("[^a-zA-Z]", " ", plain_text) #remove non-alphabets
    words = letters.lower().split()
    stopset = set(stopwords.words('english')) #searching a set is faster than searching a list
    required_words = [word for word in words if not word in stopset ]
    return (" ".join(required_words))    

In [15]:
total_reviews = df.review.size
review_words = []
for i in range(0,total_reviews):
    review_words.append(raw_statement_to_words(df.review[i]))

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)

In [70]:
X = vectorizer.fit_transform(review_words)

In [71]:
X.nnz

1975048

In [25]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [72]:
nb.fit(X,df.sentiment)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [44]:
df_test = pd.read_csv('testData.tsv',header=0,delimiter='\t',quoting=3)
num_test_reviews = df_test.review.size
test_review_words = []
for i in range(0,num_test_reviews):
    test_review_words.append(raw_statement_to_words(df_test.review[i]))

2391448

In [51]:
X_test = vectorizer.transform(test_review_words)
X_test.nnz

2350196

In [53]:
predictions = nb.predict(X_test)

In [73]:
output = pd.DataFrame( data={"id":df_test["id"], "sentiment":predictions} )
output.to_csv( "IMDB_review_predictions.csv", index=False, quoting=3 )

In [74]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",0
3,"""7186_2""",0
4,"""12128_7""",1


In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
forest = RandomForestClassifier(n_estimators = 100) 

In [75]:
forest = forest.fit( X , df["sentiment"] )

In [63]:
random_forest_output = forest.predict(X_test)

In [76]:
random_forest_output = pd.DataFrame( data={"id":df_test["id"], "sentiment":predictions} )
random_forest_output.to_csv( "IMDB_review_predictions_random_forest.csv", index=False, quoting=3 )